<a href="https://colab.research.google.com/github/PeterRutkowski/fer/blob/master/data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from PIL import Image
import numpy as np
import random
import cv2

In [ ]:
path = 'drive/My Drive/project42/KDEF_and_AKDEF/KDEF/'
folders = np.sort(os.listdir(path))
print(folders)

['AF01' 'AF02' 'AF03' 'AF04' 'AF05' 'AF06' 'AF07' 'AF08' 'AF09' 'AF10'
 'AF11' 'AF12' 'AF13' 'AF14' 'AF15' 'AF16' 'AF17' 'AF18' 'AF19' 'AF20'
 'AF21' 'AF22' 'AF23' 'AF24' 'AF25' 'AF26' 'AF27' 'AF28' 'AF29' 'AF30'
 'AF31' 'AF32' 'AF33' 'AF34' 'AF35' 'AM01' 'AM02' 'AM03' 'AM04' 'AM05'
 'AM06' 'AM07' 'AM08' 'AM09' 'AM10' 'AM11' 'AM12' 'AM13' 'AM14' 'AM15'
 'AM16' 'AM17' 'AM18' 'AM19' 'AM20' 'AM21' 'AM22' 'AM23' 'AM24' 'AM25'
 'AM26' 'AM27' 'AM28' 'AM29' 'AM30' 'AM31' 'AM32' 'AM33' 'AM34' 'AM35'
 'BF01' 'BF02' 'BF03' 'BF04' 'BF05' 'BF06' 'BF07' 'BF08' 'BF09' 'BF10'
 'BF11' 'BF12' 'BF13' 'BF14' 'BF15' 'BF16' 'BF17' 'BF18' 'BF19' 'BF20'
 'BF21' 'BF22' 'BF23' 'BF24' 'BF25' 'BF26' 'BF27' 'BF28' 'BF29' 'BF30'
 'BF31' 'BF32' 'BF33' 'BF34' 'BF35' 'BM01' 'BM02' 'BM03' 'BM04' 'BM05'
 'BM06' 'BM07' 'BM08' 'BM09' 'BM10' 'BM11' 'BM12' 'BM13' 'BM14' 'BM15'
 'BM16' 'BM17' 'BM18' 'BM19' 'BM20' 'BM21' 'BM22' 'BM23' 'BM24' 'BM25'
 'BM26' 'BM27' 'BM28' 'BM29' 'BM30' 'BM31' 'BM32' 'BM33' 'BM34' 'BM35']


In [ ]:
data, labels = [], []

counter = 0
for folder in folders:
  images = np.sort(os.listdir('{}{}/'.format(path, folder)))
  frontal_images = []
  for image in images:
    if image[6] == 'S':
      img = np.asarray(Image.open('{}{}/{}'.format(path, folder, image)))
      gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      data.append(gray)
      labels.append(image[4:6])

data = np.asarray(data)
labels = np.asarray(labels)

np.savez_compressed('drive/My Drive/project42/KDEF.npz', x=data, y=labels)

In [ ]:
loaded = np.load('drive/My Drive/project42/KDEF.npz', allow_pickle=True)
X = loaded['x']
Y = loaded['y']

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

class FaceDetector:
    def __init__(self):
        self.__HaarCascade = cv2.CascadeClassifier('drive/My Drive/project42/haarcascade_frontalface_default.xml')
        self.faces = np.empty(0)
        self.img = np.empty(0)

    def print_with_boundaries(self):
        assert self.faces.size, 'DetectorAssert: no faces @ print_face_boundaries'
        assert self.img.size, 'DetectorAssert : no image @ print_face_boundaries'
        img = self.img
        for rectangle in self.faces:
            x, y, w, h = rectangle
            img = cv2.rectangle(self.img, (x, y), (x+w,y+h), (255,0,0), 2)
        cv2_imshow(img)


    def __grayscale(self):
        assert self.img.size, 'DetectorAssert: no image @ __greyscale'
        self.img = cv2.cvtColor(self.img, cv2.COLOR_BGR2GRAY)

    def detect(self, img=None, scaleFactor=1.2, minNeghbors=4):
        if img is not None:
            self.img = img

        assert self.img.size, 'DetectorAssert : no image @ detect'
        detector = self.__HaarCascade
        self.faces = detector.detectMultiScale(self.img, scaleFactor, minNeghbors)

    def extract_faces(self):
        cropped = []
        for face in self.faces:
            x, y, w, h = face
            cropped.append(self.img[y:y+h, x:x+w])
        return cropped

In [ ]:
fd = FaceDetector()
scaled_faces = []
for i in range(len(X)):
  fd.detect(X[i])
  x,y,w,h,=fd.faces[0]
  fd.img = fd.img[y:y+h,x:x+w]
  fd.img = cv2.resize(fd.img, (100,100))
  scaled_faces.append(fd.img)

scaled_faces = np.asarray(scaled_faces)

np.savez_compressed('drive/My Drive/project42/KDEF_NN.npz', x=scaled_faces, y=Y)

In [2]:
loaded = np.load('drive/My Drive/project42/KDEF_NN.npz', allow_pickle=True)
X = loaded['x']
Y = loaded['y']

In [3]:
classes = np.unique(Y)
print(classes)
dict_classes = {}
for i in range(len(Y)):
  if Y[i] in dict_classes:
    dict_classes[Y[i]][0] += 1
    dict_classes[Y[i]][1].append(i)
  else:
    dict_classes[Y[i]] = [1,[i]]

print(dict_classes)

['AF' 'AN' 'DI' 'HA' 'NE' 'SA' 'SU']
{'AF': [141, [0, 7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 91, 98, 105, 112, 119, 126, 133, 140, 147, 154, 161, 168, 175, 182, 189, 196, 203, 210, 217, 224, 231, 238, 245, 252, 259, 266, 273, 280, 287, 294, 301, 308, 315, 322, 329, 336, 343, 350, 357, 364, 371, 378, 385, 392, 399, 406, 413, 420, 427, 434, 441, 448, 455, 462, 469, 476, 483, 490, 497, 504, 511, 518, 525, 532, 539, 546, 553, 560, 567, 574, 581, 588, 595, 602, 609, 616, 623, 630, 637, 644, 651, 658, 665, 672, 679, 680, 687, 694, 701, 708, 715, 722, 729, 736, 743, 750, 757, 764, 771, 778, 785, 792, 799, 806, 813, 820, 827, 834, 841, 848, 855, 862, 869, 876, 883, 890, 897, 904, 911, 918, 925, 932, 939, 946, 953, 960, 967, 974]], 'AN': [140, [1, 8, 15, 22, 29, 36, 43, 50, 57, 64, 71, 78, 85, 92, 99, 106, 113, 120, 127, 134, 141, 148, 155, 162, 169, 176, 183, 190, 197, 204, 211, 218, 225, 232, 239, 246, 253, 260, 267, 274, 281, 288, 295, 302, 309, 316, 323, 330, 337, 344, 351, 358, 365

In [5]:
import random
x_train, y_train, x_val, y_val = [], [], [], []
class_quantity = 140
val_split = 0.3

for el in dict_classes.keys():
  val_ind = random.sample(range(class_quantity), int(class_quantity*val_split))
  for i in range(class_quantity):
    if i in val_ind:
      x_val.append(X[i])
      y_val.append(Y[i])
    else:
      x_train.append(X[i])
      y_train.append(Y[i])

x_train = np.asarray(x_train)
y_train = np.asarray(y_train)
x_val = np.asarray(x_val)
y_val = np.asarray(y_val)

In [8]:
np.savez_compressed('drive/My Drive/project42/KDEF_F.npz', x_train=x_train, y_train=y_train, x_val=x_val, y_val=y_val)